In [135]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import streamlit as st

In [160]:
import streamlit as st

# Streamlit app
st.title('Book Recommendation System')

# Select user
user_id = st.number_input('Enter User ID:', min_value=10, value=10)

# Generate recommendations
if st.button('Recommend Books'):
    try:
        recommendations = get_recommendations(user_id)
        st.write('Recommended Books:')
        for book in recommendations:
            st.write(book)
    except KeyError:
        st.write('User ID not found. Please enter a valid User ID.')

In [161]:
# Load datasets
users = pd.read_csv('Users.csv', encoding='latin-1', on_bad_lines='skip')
books = pd.read_csv('Books (1).csv', encoding='latin-1', on_bad_lines='skip')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1', on_bad_lines='skip')

C:\Users\Deepak\AppData\Local\Temp\ipykernel_8656\3607546377.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books (1).csv', encoding='latin-1', on_bad_lines='skip')


In [162]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [163]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [164]:
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [165]:
# Check column names in each dataset
print("Users columns:", users.columns)
print("Books columns:", books.columns)
print("Ratings columns:", ratings.columns)

Users columns: Index(['User-ID', 'Location', 'Age'], dtype='object')
Books columns: Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Ratings columns: Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [166]:
books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
users.drop("Location",axis=1,inplace=True)

In [167]:
books['Book-Author'].fillna('Unknown_Author',inplace=True)
books["Publisher"].fillna('Unknown_Publisher',inplace=True)
users['Age'].fillna(users.Age.mean(),inplace=True)

In [168]:
# Merge datasets
data = ratings.merge(books, on="ISBN", how="inner")
data.reset_index(drop=True, inplace=True)

In [169]:
data

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA


In [170]:
# Remove duplicates
data['ISBN'].drop_duplicates

<bound method Series.drop_duplicates of 0          034545104X
1          034545104X
2          034545104X
3          034545104X
4          034545104X
              ...    
1031131    0517145553
1031132    1575660792
1031133    0590907301
1031134    0679752714
1031135    0806917695
Name: ISBN, Length: 1031136, dtype: object>

In [171]:
data["User-ID"].isnull().sum()

0

In [172]:
data["Book-Title"].isnull().sum()

0

In [173]:
# Check the index of your DataFrame
print(data.index)

RangeIndex(start=0, stop=1031136, step=1)


In [174]:
print(data.index.duplicated().any())


False


In [175]:
# users who have rated more than 200 times
x = data.groupby('User-ID').count()['Book-Rating'] > 200
educated_users = x[x].index
educated_users

Int64Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,
              6323,   6543,
            ...
            271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427,
            277639, 278418],
           dtype='int64', name='User-ID', length=811)

In [176]:
filtered_rating = data[data['User-ID'].isin(educated_users)]
filtered_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
5,23768,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
7,28523,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
15,77940,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
16,81977,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
...,...,...,...,...,...,...,...
1030883,275970,1880837927,0,The Theology of the Hammer,Millard Fuller,1994,Smyth &amp; Helwys Publishing
1030884,275970,188717897X,0,The Ordeal of Integration: Progress and Resent...,Orlando Patterson,1998,Civitas Book Publisher
1030885,275970,1888889047,0,Pushcart's Complete Rotten Reviews &amp; Rejec...,Bill Henderson,1998,Pushcart Press
1030886,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA)


In [177]:
# books which have been rated more than 50 times
y = filtered_rating.groupby('Book-Title').count()['Book-Rating'] >= 50
famous_books = y[y].index

In [178]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]
final_ratings

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books
74,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books
...,...,...,...,...,...,...,...
1026724,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub
1027923,269566,0670809381,0,Echoes,Maeve Binchy,1986,Penguin USA
1028777,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island
1029070,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster


In [179]:
# drop the duplicate values if any
final_ratings.drop_duplicates()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books
69,11676,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books
74,23768,0446520802,6,The Notebook,Nicholas Sparks,1996,Warner Books
...,...,...,...,...,...,...,...
1026724,266865,0531001725,10,The Catcher in the Rye,Jerome David Salinger,1973,Scholastic Library Pub
1027923,269566,0670809381,0,Echoes,Maeve Binchy,1986,Penguin USA
1028777,271284,0440910927,0,The Rainmaker,John Grisham,1995,Island
1029070,271705,B0001PIOX4,0,Fahrenheit 451,Ray Bradbury,1993,Simon &amp; Schuster


In [180]:
# contains books which have been rated atleast 50 times and users who have rated more than 200 books
user_item_matrix = final_ratings.pivot_table(index='Book-Title', columns = 'User-ID', values = 'Book-Rating')
user_item_matrix

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Bend in the Road,0.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,9.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [181]:
# replace NaN with 0
user_item_matrix.fillna(0,inplace=True)

In [182]:
# Calculate cosine similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Streamlit app
st.title('Book Recommendation System')

# Select user
user_id = st.number_input('Enter User ID:', min_value=int(users['User-ID'].min()), value=int(users['User-ID'].min()))

# Generate recommendations
if st.button('Recommend Books'):
    recommendations = get_recommendations(user_id)
    if recommendations:
        st.write('Recommended Books:')
        for book in recommendations:
            st.write(book)
    else:
        st.write('User ID not found. Please enter a valid User ID.')


In [183]:
def get_recommendations(user_id, num_recommendations=5):
    if user_id not in user_similarity_df.index:
        return None
    
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:num_recommendations+1]
    similar_user_ids = similar_users.index
    similar_user_books = user_item_matrix.loc[similar_user_ids].mean(axis=0)
    recommended_books = similar_user_books.sort_values(ascending=False).head(num_recommendations)
    return recommended_books.index.tolist()
